 # Iteration 02 Data Wrangling - On Your Toes (Part 2) 

Last Update : 14 May 2020
Version : 1.0

Data Sets analysed: Indooor Activities

* Grid view.csv 
* edited_exercises.csv (Manually filtered version of the "Grid view.csv" file, based on research )

In this notebook, Crime datasets used in the On Your Toes - The fitness app are put through data wrangling tasks to inspect data, clean up and create custom datasets to be exported to the MySQL database.


Detailed data plan :https://mahara.infotech.monash.edu.au/mahara/artefact/file/download.php?file=239816

-----------------------------------------------------------------

#### Import Libraries and Packages

In [1]:
import pandas as pd
import numpy as np
import re


#### Import datasets

In [2]:
dataset = pd.read_csv("Grid view.csv")
dataset.head(3)

,Exercise,Equipment,Exercise Type,Major Muscle,Minor Muscle,Example,Notes,Modifications
0,Arnold Press,Dumbbells,Weight,Arms,"Bicep,Shoulders",giphy-2.gif (https://dl.airtable.com/FvaObZ1Sy...,NaN,NaN
1,Bicep Curl,"Bar,Dumbbells,Cable","Weight,Machine",Arms,Bicep,3e8b7121-738e-4457-b6d8-fd3a3e04de5a.gif (http...,Don't use momentum. Make sure the movement is ...,NaN
2,Bicycle Crunch,Body Weight,Weight,Core,NaN,200.gif (https://dl.airtable.com/Y0JUbM2YTfe8u...,"The lower the ""straight"" leg is to the ground ...","Easier: Move Slower, Legs higher in the air\nH..."


In [3]:
edited_dataset = pd.read_csv("edited_exercises.csv")
edited_dataset.head(3)

,Exercise,Equipment,Exercise Type,Muscle Group,Notes,Modifications,Alt Equip,Sets,Reps
0,Arnold Press,Dumbbells,Weight,"Arms - Bicep,Shoulders",NaN,NaN,"Filled water bottle 1L = 1Kg, band",2-3,10-12
1,Bicep Curl,"Bar,Dumbbells,Cable","Weight,Machine",Arms -Bicep,Don't use momentum. Make sure the movement is ...,NaN,Filled water bottle 1L = 1Kg,2-3,10-15
2,Bicycle Crunch,Body Weight,Weight,Core,"The lower the ""straight"" leg is to the ground ...","Easier: Move Slower, Legs higher in the air\nH...",NaN,3 -4,10-15


#### Merge the filtered datatset with original dataset to include the weblinks of exercise gifs

In [4]:
exercises_export = pd.merge(edited_dataset, dataset, how='left', on=['Exercise', 'Exercise'])
len(exercises_export)

44

#### Remove unnecessary columns form the dataset

In [5]:
exercises_export.columns

Index(['Exercise', 'Equipment_x', 'Exercise Type_x', 'Muscle Group', 'Notes_x',
       'Modifications_x', 'Alt Equip', 'Sets', 'Reps', 'Equipment_y',
       'Exercise Type_y', 'Major Muscle', 'Minor Muscle', 'Example', 'Notes_y',
       'Modifications_y'],
      dtype='object')

In [6]:
exercises_export=exercises_export[['Exercise', 'Equipment_x', 'Exercise Type_x', 'Muscle Group', 'Notes_x',
       'Modifications_x', 'Alt Equip', 'Sets', 'Reps','Example']]

#### Reanme columns affected by the merge statement in previos steps

In [7]:
exercises_export.rename(columns={'Equipment_x':'Equipment','Exercise Type_x':'ExerciseType','Notes_x':'Notes','Modifications_x':'Modifications','Alt Equip':'AlternateEquipments', 'Muscle Group':'MuscleGroup'}, inplace=True)
exercises_export.head()

,Exercise,Equipment,ExerciseType,MuscleGroup,Notes,Modifications,AlternateEquipments,Sets,Reps,Example
0,Arnold Press,Dumbbells,Weight,"Arms - Bicep,Shoulders",NaN,NaN,"Filled water bottle 1L = 1Kg, band",2-3,10-12,giphy-2.gif (https://dl.airtable.com/FvaObZ1Sy...
1,Bicep Curl,"Bar,Dumbbells,Cable","Weight,Machine",Arms -Bicep,Don't use momentum. Make sure the movement is ...,NaN,Filled water bottle 1L = 1Kg,2-3,10-15,3e8b7121-738e-4457-b6d8-fd3a3e04de5a.gif (http...
2,Bicycle Crunch,Body Weight,Weight,Core,"The lower the ""straight"" leg is to the ground ...","Easier: Move Slower, Legs higher in the air\nH...",NaN,3 -4,10-15,200.gif (https://dl.airtable.com/Y0JUbM2YTfe8u...
3,Bounds,Body Weight,"Cardio,Laps","Full Body,Legs - Outer Thigh,Glutes",Do laps across the room of these.,Easier: Don't jump as high/far\nHarder: Touch ...,NaN,2-3,10-15,1ea07303-2e87-4876-8e68-8944edd42173.gif (http...
4,Box Toe Touch,Platform,Cardio,Legs - Glutes,Take care when doing the activity,"Easier: Don't use a box, just practice toe tou...","Chair, Climbing stairs",1-2,8-10,workoutanniversarygif9570.gif (https://dl.airt...


#### Filter in the bodyweight exercises to include in the final dataset

This was done because travellers have limited access to specific exercise instruments in their accomodations

In [8]:
exercises_export = exercises_export[exercises_export['Equipment'].isin(['Body Weight','Body Weight,Bosu Ball'])]
exercises_export.head(3)

,Exercise,Equipment,ExerciseType,MuscleGroup,Notes,Modifications,AlternateEquipments,Sets,Reps,Example
2,Bicycle Crunch,Body Weight,Weight,Core,"The lower the ""straight"" leg is to the ground ...","Easier: Move Slower, Legs higher in the air\nH...",NaN,3 -4,10-15,200.gif (https://dl.airtable.com/Y0JUbM2YTfe8u...
3,Bounds,Body Weight,"Cardio,Laps","Full Body,Legs - Outer Thigh,Glutes",Do laps across the room of these.,Easier: Don't jump as high/far\nHarder: Touch ...,NaN,2-3,10-15,1ea07303-2e87-4876-8e68-8944edd42173.gif (http...
5,Burpee,"Body Weight,Bosu Ball",Plyo,Full Body,"Option with half Bosu, Pushup Optional. Make s...","Easier: Don't Jump after, and break down motio...",NaN,1-2,10-12,Burpee.gif (https://dl.airtable.com/xDZ3bhDQqG...


#### Remove the unnecessary columns in the dataset

In [9]:
exercises_export = exercises_export[['Exercise','ExerciseType','MuscleGroup','Modifications','Sets','Reps','Example']]


#### Extract hyperlinks from the Example column using regular expression

In [10]:
exercises_export['Example'] = exercises_export['Example'] .str.extract('\(([^)]+)\)')

#### Replace Nan with None

In [11]:
exercises_export = exercises_export.where(pd.notnull(exercises_export), 'None')
exercises_export = exercises_export.reset_index(drop=True)

#### Export dataset

In [12]:
exercises_export.to_csv('Exercises_Export.csv',index= False)


---------------------------------

#### Export file produced : `Exercises_Export.csv`

## End of data wrangling process for iteration 02 (Part 2)